# Supplementary Figure S2: Comparison of meteo conditions

In [ ]:
import pandas as pd
import numpy as np
import glob
import re

from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format
# Colours
cbPalette = ['#939393', '#0072B2', '#E69F00', '#CC00CC', '#009E73', '#D55E00', '#CC79A7', '#FF3300', '#F0E442', '#56B4E9']

In [ ]:
project_path = '../'
data_path   = project_path + '../data/'

# Main dataset
main_dataset_fn = data_path + 'dataset.csv'

# Auxiliary meteo data
meteo_pv_fn  = data_path + 'met_data_pv.csv'
meteo_aff_fn = data_path + 'afforestation_albedo.csv'

# Outputs
graphs_path  = project_path + '../graphs/'

### Functions

In [ ]:
def load_data(fn, index_col=False, silent=False):
    if (not silent):
        print('-', fn.split('/')[-1])
    temp = pd.read_csv(fn, index_col=index_col)
    temp['DateTime'] = pd.to_datetime(temp['DateTime'], format='%Y-%m-%d %H:%M:%S')
    return(temp)

# Create annual daily values
def make_daily_annual(temp, groups=[]):
    temp = temp.copy()
    temp['Day'] = temp['DateTime'].dt.strftime('%m-%d')
    annual_df = temp.groupby(['Day'] + groups).mean()
    annual_df.reset_index(inplace = True)
    annual_df['DateTime'] = pd.to_datetime(annual_df['Day'], format='%m-%d')
    return(annual_df)

In [ ]:
print('Loading data...')

# Main dataset
df = load_data(main_dataset_fn)

# Meteo data (Station near PV Field)
met_pv = load_data(meteo_pv_fn)

# Meteo data (Forest research station)
met_aff = load_data(meteo_aff_fn)
met_aff['Ecosystem'] = 'Afforestation'
# Keep only the same years as truck campaigns, and relevant columns
met_aff = met_aff.loc[(met_aff['DateTime'].dt.year == 2013) | (met_aff['DateTime'].dt.year == 2015), ['DateTime','Ecosystem', 'SWin', 'Ta']]

print('Done...')

In [ ]:
# Select relevant columns
small_df = df.loc[df['Ecosystem'] != 'Afforestation', ['Ecosystem', 'DateTime', 'SWin', 'Ta']].copy().reset_index(drop=True)

# Assign location
small_df['Location'] = np.nan
small_df.loc[small_df['Ecosystem'].isin(['PV desert background', 'PV field']), 'Location']                 = 'PV Field Area (Ketura)'
small_df.loc[small_df['Ecosystem'].isin(['Afforestation desert background', 'Afforestation']), 'Location'] = 'Forest Area (Yatir)'

# Assign source
small_df['Source'] = 'Mobile Lab'

# Meteo station data at afforestation ('Forest Research Station')
met_aff['Source'] = 'Meteo Station'
met_aff['Location'] = 'Forest Area (Yatir)'

# Meteo station data at PV field
met_pv = met_pv[['Ecosystem', 'DateTime', 'SWin', 'Ta']].copy().reset_index(drop=True)
met_pv['Source'] = 'Meteo Station'
met_pv['Location'] = 'PV Field Area (Ketura)'

# Merge with meteo station data
all_df = pd.concat([small_df, met_pv], axis=0, ignore_index=True)
all_df = pd.concat([all_df, met_aff], axis=0, ignore_index=True)

# Make daily means
annual_df = make_daily_annual(all_df, groups=['Location', 'Ecosystem', 'Source'])

# Remove bad values
annual_df.loc[(annual_df['Ecosystem'].isin(['PV desert background', 'PV field'])) & (annual_df['SWin'] > 400), 'SWin'] = np.nan # Remove half-day in Ketura PV field
annual_df.loc[(annual_df['SWin'] < 10), 'SWin'] = np.nan # Remove outliers

In [ ]:
# Prepare
#--------
# Convert to long format
annual_long = pd.melt(annual_df[['DateTime', 'Source', 'Location', 'SWin', 'Ta']], id_vars=['DateTime', 'Source', 'Location'])
# Adjust labels
annual_long.loc[annual_long['variable'] == 'SWin', 'variable'] = '$S_{in}$ [$W~m^{-2}$]'
annual_long.loc[annual_long['variable'] == 'Ta', 'variable'] = '$T_{a}$ [°C]'

# Make figure
#--------------
plt = ggplot(annual_long)
plt = plt + geom_point(aes(x='DateTime', y='value', colour = 'Source'), size=0.5)
plt = plt + theme_bw()
plt = plt + theme(axis_text_x=element_text(angle=45),
                  axis_title_x=element_blank(), axis_title_y=element_blank())
plt = plt + labs(x='Month', y='$S_{in}$ [$W~m^{-2}$]', colour = 'Source')
plt = plt + scale_x_datetime(breaks=date_breaks('1 month'), labels=date_format('%b'))
plt = plt + scale_colour_manual(values=cbPalette) + scale_fill_manual(values=cbPalette)
plt = plt + facet_grid('variable ~ Location', scales='free_y')
print(plt)

#plt.save(graphs_path + 'meteo_parameters.png', width=1.6*15, height=1.6*7.5, units='cm', dpi=600)
#plt.save(graphs_path + 'meteo_parameters.pdf', width=1.6*15, height=1.6*7.5, units='cm', dpi=600)